NOTE:- With the change in spotify developer's new policy, mining for more songs has been suspended. After successfully generating 4000 songs, the code shows "SpotifyException: http status: 403, code:-1"

In [4]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import lyricsgenius
import os

# API credentials
SPOTIFY_CLIENT_ID = '7f6ed32beeb840348ca0a28f6c9263c3'
SPOTIFY_CLIENT_SECRET = 'b0ff0ff6a68d43d497b659a197908f50'
GENIUS_ACCESS_TOKEN = 'lUEnsDiu8zg843PFbBLzuCIgHQ6e0kCwCCt3LqvroUx3yw_x5Rh3GM9g4ObzrdS6'

# Initialize Spotify client
spotify_client_credentials_manager = SpotifyClientCredentials(client_id=SPOTIFY_CLIENT_ID, client_secret=SPOTIFY_CLIENT_SECRET)
sp = spotipy.Spotify(client_credentials_manager=spotify_client_credentials_manager)

# Initialize Genius client
genius = lyricsgenius.Genius(GENIUS_ACCESS_TOKEN)
genius.verbose = False
genius.remove_section_headers = True

# List of playlist IDs to fetch songs from
playlist_ids = [
    # '37i9dQZEVXbMDoHDwVN2tF',  # Global Top 50
    # '37i9dQZF1DXcBWIGoYBM5M',  # Today's Top Hits
    # '37i9dQZF1DX0XUsuxWHRQd',  # RapCaviar
    # '37i9dQZF1DX4JAvHpjipBk',  # New Music Friday
    # '37i9dQZF1DWXRqgorJj26U',  # Rock This
    # '5muSk2zfQ3LI70S64jbrX7', #top English songs of all time
    # '4es9oxfFPI49KvzWwjsFdR',  #kpop english version
    # '55a7nOLLrvh1gRE4GY7Loy', #japanese english songs
    # '3NP0xGr6G3ah05np9ZslEg', #90's-20's 
    # '37i9dQZF1DX4UtSsGT1Sbe', #80's songs
    # '0Sm64Lu6z1OK8yM3Oeo4Wx', #indie mix
    # '4Jb4PDWREzNnbZcOHPcZPy', # country songs
    # '45B1EGRDWDxLTL0t4BcVCy', #British
    # '37i9dQZF1DX0Yxoavh5qJV', #Christmas songs
    # '37i9dQZF1E39gVX9lcAPBs', #country Christmas songs
    '7MOyTfsZvC9eUSZFiFnbHn', #kids songs
    
    
    
    # Add more playlist IDs here if needed
]

def get_playlist_tracks(playlist_id):
    tracks = []
    offset = 0
    while True:
        try:
            results = sp.playlist_tracks(playlist_id, offset=offset, limit=100)
            tracks.extend(results['items'])
            if len(tracks) >= 2000 or results['next'] is None:
                break
            offset += len(results['items'])
        except Exception as e:
            print(f"Error fetching tracks from playlist {playlist_id}: {e}")
            break
    return tracks[:2000]  # Ensure we only return 2000 tracks max

def get_audio_features(track_ids):
    audio_features = []
    for i in range(0, len(track_ids), 100):
        chunk = track_ids[i:i+100]
        features = sp.audio_features(chunk)
        audio_features.extend(features)
    return audio_features

def get_lyrics(song_title, artist_name):
    try:
        song = genius.search_song(song_title, artist_name)
        if song:
            return song.lyrics
        else:
            return "Lyrics not found"
    except Exception as e:
        print(f"Error fetching lyrics for {artist_name} - {song_title}: {e}")
        return "Error fetching lyrics"

def fetch_songs(target_count=2000):
    all_tracks = []
    for playlist_id in playlist_ids:
        if len(all_tracks) >= target_count:
            break
        playlist_tracks = get_playlist_tracks(playlist_id)
        all_tracks.extend(playlist_tracks)
        print(f"Fetched {len(playlist_tracks)} tracks from playlist {playlist_id}")
        time.sleep(1)  # To avoid hitting rate limits

    all_tracks = all_tracks[:target_count]
    
    track_data = []
    track_ids = []

    for i, item in enumerate(all_tracks):
        if item is None or 'track' not in item or item['track'] is None:
            print(f"Skipping invalid track at index {i}")
            continue
        
        track = item['track']
        if 'id' not in track:
            print(f"Skipping track without ID at index {i}")
            continue

        track_ids.append(track['id'])
        
        # Fetch lyrics
        lyrics = get_lyrics(track['name'], track['artists'][0]['name'])
        
        # Get artist info
        artist_info = sp.artist(track['artists'][0]['id'])
        
        track_info = {
            'Title': track['name'],
            'Artist': track['artists'][0]['name'],
            'Album': track['album']['name'],
            'Release_Date': track['album']['release_date'],
            'Available_Markets': ', '.join(track.get('available_markets', [])),
            'Genre': ', '.join(artist_info.get('genres', [])),
            'ISRC': track.get('external_ids', {}).get('isrc', 'Unknown'),
            'Lyrics': lyrics,
            'Track_Popularity': track.get('popularity', 'N/A'),
            'Duration (min:sec)': f"{track['duration_ms'] // 60000}:{(track['duration_ms'] % 60000) // 1000:02d}",
            'Duration (ms)': track['duration_ms'],
            'Artist Followers': artist_info['followers']['total']
        }
        track_data.append(track_info)
        
        print(f"Processed track {i+1}/{len(all_tracks)}: {track['name']} by {track['artists'][0]['name']}")
        time.sleep(1)  # Add delay between requests

    print("Fetching audio features...")
    audio_features = get_audio_features(track_ids)

    for i, features in enumerate(audio_features):
        if features:
            track_data[i].update({
                'Tempo': features['tempo'],
                'Key': features['key'],
                'Mode': features['mode'],
                'Danceability': features['danceability'],
                'Energy': features['energy'],
                'Instrumentalness': features['instrumentalness'],
                'Acousticness': features['acousticness'],
                'Speechiness': features['speechiness'],
                'Valence': features['valence'],
                'Loudness': features['loudness'],
            })

    return pd.DataFrame(track_data)

# Fetch songs and create DataFrame
df = fetch_songs(target_count=2000)

# Reorder columns to match the desired output
column_order = [
    'Title', 'Artist', 'Album', 'Release_Date', 'Available_Markets', 'Genre', 'ISRC', 'Lyrics',
    'Tempo', 'Key', 'Mode', 'Danceability', 'Energy', 'Instrumentalness', 'Acousticness',
    'Speechiness', 'Valence', 'Loudness', 'Track_Popularity', 'Duration (min:sec)', 'Duration (ms)',
    'Artist Followers'
]
df = df.reindex(columns=column_order)

# Save to CSV, appending if file exists
file_path = 'spotify_songs_with_lyrics.csv'

if os.path.exists(file_path):
    # File exists, append without writing the header
    df.to_csv(file_path, mode='a', header=False, index=False)
    print(f"Appended {len(df)} new songs to {file_path}")
else:
    # File doesn't exist, create it and write the header
    df.to_csv(file_path, index=False)
    print(f"Created new file and saved {len(df)} songs to {file_path}")

print("Data collection and saving complete!")

Fetched 77 tracks from playlist 7MOyTfsZvC9eUSZFiFnbHn
Processed track 1/77: Baby Shark by Pinkfong
Processed track 2/77: Old MacDonald Had a Farm by The ABC Academy
Processed track 3/77: I Have A Pet by Super Simple Songs
Processed track 4/77: Ten In the Bed by Super Simple Songs
Processed track 5/77: Splish Splash by Miny & Moe
Processed track 6/77: Itsy Bitsy Spider by Piper
Processed track 7/77: Skidamarink by CoComelon
Processed track 8/77: If You're Happy and You Know It by The ABC Academy
Processed track 9/77: One Potato, Two Potatoes by CoComelon
Processed track 10/77: London Bridge is Falling Down by Little Baby Bum Nursery Rhyme Friends
Processed track 11/77: B-I-N-G-O by Piper
Processed track 12/77: Teddy Wants To Cuddle With You by Miny & Moe
Processed track 13/77: London Bridge is Falling Down by Piper
Processed track 14/77: I'm A Choo Choo Train by Kibbies
Processed track 15/77: Shake and Move by Patty Shukla
Processed track 16/77: Mary Had a Little Lamb by Piper
Processe

HTTP Error for GET to https://api.spotify.com/v1/audio-features/?ids=5ygDXis42ncn6kYG14lEVG,7d4k4uepd9d24OAE1jaafa,5P0PXe4jbM0IXnhovqIJD8,50yuj2IllSKWpaVqw08DLQ,1TnPhodJvcqCCROATfGE28,0UfeXVWqTapgLRcSAS2x2E,3WSrard3x0Mti9cHysN6jA,6t7wrhS9tDi76ZfRGxjB3D,1YrTNjzCFkO9qS7hdQxpqM,2xdmCshm23eC8oIaxPD6su,5GdG0m9SxedjhOAMQknpL8,4Nz7dmfPaPNbTjGR97nKQ9,4qYXwbtUjfdzTSJG1iWSx7,3pIZ4Wj36eF3clCOVD7Lqr,2tZUOYK2y7WNhArsa3oxUn,7tYu5KhLhVuyQiECaQ2fWb,1ebRcxynQDI3O55S3V2K9P,4Bkx63q4AmXe2d4lF7zERJ,0Ujun32VQNIZQiSZy4tTju,0I573IAvhHnoW9TchprsbB,6M3HhWR5Yl7uhXy16gkPyb,6T15qWkGDGlE9vTkvoT4oZ,3I5wtrOSSZZ89n2iOLMHZI,1gPop7Y9JYA2F7sn62RMPP,7N0aFr8qHZyNRW6H2OIQeY,1Hdf1j7KhCm3tyi57ckcDn,5qtSmNA79JgaxFXqFjRmcG,7Ca5v3kt4wrtvuX4Q5nIjy,2gz3Vz96Em9zE10KJ558OL,3Lb6dD3UO3q83PBV4fLheF,6ivfiv3piJAO7S7Ct81WGf,6d2DK7psYkREOWbBtYXKpk,6en2TbTKRWrLiItB0qn74f,3PpuDg0ZVPviHuljT3R087,1fgKsdPNOj8QGPpFfXQhz3,4ccFaisUok6s0vT1wbXXDz,5I6VOMgBaxwcUMTF4L1bXu,7hZPVyXTSxuEHFeBNjdGl1,1gpVTO4EAbeTgi7Xu9H14k,5JOzW6OSiq3T9UO7TMFlQF,2R5iEqOhOie

Fetching audio features...


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/audio-features/?ids=5ygDXis42ncn6kYG14lEVG,7d4k4uepd9d24OAE1jaafa,5P0PXe4jbM0IXnhovqIJD8,50yuj2IllSKWpaVqw08DLQ,1TnPhodJvcqCCROATfGE28,0UfeXVWqTapgLRcSAS2x2E,3WSrard3x0Mti9cHysN6jA,6t7wrhS9tDi76ZfRGxjB3D,1YrTNjzCFkO9qS7hdQxpqM,2xdmCshm23eC8oIaxPD6su,5GdG0m9SxedjhOAMQknpL8,4Nz7dmfPaPNbTjGR97nKQ9,4qYXwbtUjfdzTSJG1iWSx7,3pIZ4Wj36eF3clCOVD7Lqr,2tZUOYK2y7WNhArsa3oxUn,7tYu5KhLhVuyQiECaQ2fWb,1ebRcxynQDI3O55S3V2K9P,4Bkx63q4AmXe2d4lF7zERJ,0Ujun32VQNIZQiSZy4tTju,0I573IAvhHnoW9TchprsbB,6M3HhWR5Yl7uhXy16gkPyb,6T15qWkGDGlE9vTkvoT4oZ,3I5wtrOSSZZ89n2iOLMHZI,1gPop7Y9JYA2F7sn62RMPP,7N0aFr8qHZyNRW6H2OIQeY,1Hdf1j7KhCm3tyi57ckcDn,5qtSmNA79JgaxFXqFjRmcG,7Ca5v3kt4wrtvuX4Q5nIjy,2gz3Vz96Em9zE10KJ558OL,3Lb6dD3UO3q83PBV4fLheF,6ivfiv3piJAO7S7Ct81WGf,6d2DK7psYkREOWbBtYXKpk,6en2TbTKRWrLiItB0qn74f,3PpuDg0ZVPviHuljT3R087,1fgKsdPNOj8QGPpFfXQhz3,4ccFaisUok6s0vT1wbXXDz,5I6VOMgBaxwcUMTF4L1bXu,7hZPVyXTSxuEHFeBNjdGl1,1gpVTO4EAbeTgi7Xu9H14k,5JOzW6OSiq